In [ ]:
import pandas as pd

In [ ]:
# read in dataframe
message_filename = "/Users/gandalf/Documents/coding/data_do_not_commit/raw_data_messages.json"
DF = pd.read_json(message_filename)
print("... read in dataframe")

In [ ]:
# create a copy
df = DF.copy()
df = df[24:]
print("... created a copy and cropped odd convos from start")

In [ ]:
# split into conversation and last message dataframes
df['conv_flag'] = df.conversations.apply(lambda x: 1 if isinstance(x, dict) else 0)
convo_df = df[df.conv_flag==1].drop(['users','conv_flag'],axis=1)
users_df = df[df.conv_flag==0].drop(['conversations','conv_flag'],axis=1)
print("... split into conversation and last message dataframes")

In [ ]:
# get conversation length
convo_df['len_convo'] = convo_df.conversations.apply(len)
print("... got conversation lengths")

In [ ]:
# to get all possible column headings (#oneliners)
master_set = set()
for s in convo_df.conversations.apply(lambda x: set(list(x.values())[0].keys())).tolist(): master_set = master_set.union(s) 

def get_deets_fn(x,label):
    try: return list(x.values())[0][label]
    except: return None
    
column_headings = list(master_set)

for heading in column_headings:
    convo_df['first_message_' + heading] = convo_df.conversations.apply(lambda x: get_deets_fn(x,heading))
print("... got details from the first message")

In [ ]:
# this is really gross, but has a kinda nifty end if you're willing to invest the time

convo_df['indexies'] = convo_df.index
convo_df['first_ten'] = convo_df.indexies.apply(lambda x: x[:10])
convo_df['second_ten'] = convo_df.indexies.apply(lambda x: x[10:])
convo_df['is_first'] = convo_df.first_message_uid == convo_df['first_ten']
convo_df['is_second'] = convo_df.first_message_uid == convo_df['second_ten']
convo_df['second_message_uid'] = convo_df.is_second*convo_df.first_ten + convo_df.is_first*convo_df.second_ten

print("... added second message uid")

In [ ]:
# get len of message text
convo_df['first_message_len'] = convo_df.first_message_text.fillna('').apply(len)
print("... got the length of the first message")

In [ ]:
# other cleanup
convo_df.first_message_timestamp = \
    convo_df.first_message_timestamp.apply(lambda x: 
                                           pd.to_datetime(x*1000000) 
                                           if isinstance(x, int) 
                                           else x)
print("... converted timestamp to datetime")

In [ ]:
# start droppin
col_to_drop = ['conversations','indexies','first_ten','second_ten','is_first','is_second']
convo_df.drop(col_to_drop,axis=1)
print("... dropped a bunch of lame columns")